# import libraries

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime,date
import geopandas as gpd
from geopandas import GeoDataFrame as gdf
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from shapely.geometry import Point
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime

import glob
import os

# load data

In [4]:
raw_data = pd.read_csv('..\Raw GPS data Kandy Buses\digana_2022_10.csv')
bus_terminals= pd.read_csv('bus_stops_and_terminals_654.csv')

In [5]:
raw_data

,id,deviceid,devicetime,latitude,longitude,speed
0,1453607164,116,2022-09-30 23:56:32,8.222097,80.522697,0.0000
1,1453607165,116,2022-09-30 23:56:47,8.222097,80.522697,0.0000
2,1453607166,116,2022-09-30 23:57:02,8.222097,80.522697,0.0000
3,1453607167,116,2022-09-30 23:57:13,8.221493,80.522667,16.1987
4,1453607168,116,2022-09-30 23:57:28,8.219942,80.522738,24.2981
...,...,...,...,...,...,...
978912,1550050726,123,2022-11-01 22:54:40,7.308682,80.766712,11.3391
978913,1550050727,123,2022-11-01 22:54:25,7.307783,80.767427,20.5184
978914,1550050728,123,2022-11-01 22:54:10,7.306388,80.767842,17.2786
978915,1550050729,123,2022-11-01 22:53:55,7.305193,80.767347,15.1188


In [6]:
raw_data.shape

(978917, 6)

In [7]:
# df = raw_data[raw_data['deviceid']==123]
# df['devicetime'] = pd.to_datetime(df['devicetime'])
# df = df[(df['devicetime'] >= '2021-10-04 11:01:04') & (df['devicetime'] <= '2021-10-04 11:50:47')]
# df

# data cleaning

In [8]:
def raw_data_cleaning(raw_data):
  #raw_data = raw_data.drop(drop_columns, axis = 1)

  gps_data = raw_data[raw_data.latitude != 0]
  gps_data = gps_data[gps_data.longitude != 0] #cleaning zero values for latitude & longitude

  gps_data['date'] = pd.to_datetime(gps_data['devicetime']).dt.date #split date and time separately into datetime variables
  gps_data['time'] = pd.to_datetime(gps_data['devicetime']).dt.time
  gps_data['date'] = pd.to_datetime(gps_data['date'])
  
  gps_data = gps_data.sort_values(['date', 'time']) #sorting dataset by time and device

  # additional_columns = ['servertime','fixtime','address','routeid']

  # gps_data = gps_data.drop(columns=additional_columns, axis = 1) 

  gps_data.sort_values(by=['devicetime'], inplace=True) #sorting dataset by time and device

  # remove wrong dates
  gps_data = gps_data[gps_data['date'] >=  '2022-10-01']
  gps_data = gps_data[gps_data['date'] <= '2022-10-31'] 


  # drop 'date' and 'time' columns
  gps_data = gps_data.drop(columns=['date', 'time'], axis = 1)


  gps_data.reset_index(drop = True, inplace = True)

  return gps_data

In [9]:
raw_data

,id,deviceid,devicetime,latitude,longitude,speed
0,1453607164,116,2022-09-30 23:56:32,8.222097,80.522697,0.0000
1,1453607165,116,2022-09-30 23:56:47,8.222097,80.522697,0.0000
2,1453607166,116,2022-09-30 23:57:02,8.222097,80.522697,0.0000
3,1453607167,116,2022-09-30 23:57:13,8.221493,80.522667,16.1987
4,1453607168,116,2022-09-30 23:57:28,8.219942,80.522738,24.2981
...,...,...,...,...,...,...
978912,1550050726,123,2022-11-01 22:54:40,7.308682,80.766712,11.3391
978913,1550050727,123,2022-11-01 22:54:25,7.307783,80.767427,20.5184
978914,1550050728,123,2022-11-01 22:54:10,7.306388,80.767842,17.2786
978915,1550050729,123,2022-11-01 22:53:55,7.305193,80.767347,15.1188


In [10]:
gps_data = raw_data_cleaning(raw_data)

In [11]:
# unique device id in the dataset
gps_data['deviceid'].unique()

array([ 116,  279,  275,  121,  250, 1143,  123,  505, 1358, 1166,  117,
        550,  264,  513, 1377,  274,  209,  262,  128,  284], dtype=int64)

In [235]:
gps_data

,id,deviceid,devicetime,latitude,longitude,speed
0,1453608257,116,2022-10-01 00:00:13,8.195475,80.525735,26.9979
1,1453608258,116,2022-10-01 00:00:28,8.193405,80.525917,30.2376
2,1453608259,116,2022-10-01 00:00:43,8.191373,80.526890,31.8575
3,1453608260,116,2022-10-01 00:00:58,8.189095,80.527400,34.5572
4,1453608261,116,2022-10-01 00:01:13,8.186705,80.527848,35.0972
...,...,...,...,...,...,...
932684,1546710487,275,2022-10-31 23:44:27,7.278517,80.721148,0.0000
932685,1546774506,121,2022-10-31 23:45:52,7.296628,80.715947,0.0000
932686,1547254851,279,2022-10-31 23:46:37,7.294893,80.735373,0.0000
932687,1547214308,250,2022-10-31 23:48:46,7.282612,80.697462,0.0000


# preprocess bus terminals

In [236]:
bus_terminals = gpd.GeoDataFrame(bus_terminals, geometry=gpd.points_from_xy(bus_terminals.longitude,bus_terminals.latitude),crs='EPSG:4326') 

In [237]:
#  code to stop_id BT01 and BT02 and a 150m buffer and for other stop_id add 100m buffer using geometry.buffer variable
bus_terminals_buffer = bus_terminals.copy().to_crs('EPSG:5234')
bus_terminals_buffer['geometry'] = bus_terminals_buffer.geometry.buffer(100)


In [238]:
# divide bus_terminal into 2 datasets where first one has the first 16 rows and second one has the remianing rows
bus_terminals_KD = bus_terminals_buffer.iloc[1:15]
bus_terminals_DK = bus_terminals_buffer.iloc[17:]

# take only end terminals

In [239]:
end_terminal_buffer = bus_terminals_buffer[(bus_terminals_buffer['stop_id'] == 'BT01')|(bus_terminals_buffer['stop_id'] == 'BT02')]
end_terminal_buffer.geometry = end_terminal_buffer.geometry.buffer(150)

d:\Engineering\5th Semester\Project\myenv\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [240]:
bus_terminals_KD.head()

,stop_id,route_id,direction,address,latitude,longitude,geometry
1,101,654,Kandy-Digana,Wales Park,7.291186,80.637662,"POLYGON ((185076.961 232111.967, 185076.479 23..."
2,102,654,Kandy-Digana,Mahamaya,7.287840,80.645840,"POLYGON ((185979.853 231741.696, 185979.372 23..."
3,103,654,Kandy-Digana,Lewella junction,7.294430,80.650030,"POLYGON ((186442.694 232470.307, 186442.213 23..."
4,104,654,Kandy-Digana,Talwatta,7.286701,80.660336,"POLYGON ((187580.385 231615.322, 187579.903 23..."
5,105,654,Kandy-Digana,Tennekumbura Bridge,7.281866,80.666030,"POLYGON ((188209.005 231080.506, 188208.523 23..."


# filter device id 123

In [241]:
df123 = gps_data[gps_data.deviceid==123]
df123.reset_index(inplace=True)
df123.drop(columns=['index'], inplace=True)
df123

C:\Users\MSI\AppData\Local\Temp\ipykernel_8212\1241724620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df123.drop(columns=['index'], inplace=True)


,id,deviceid,devicetime,latitude,longitude,speed
0,1453659517,123,2022-10-01 03:13:43,7.296587,80.735997,0.0000
1,1453659515,123,2022-10-01 03:14:03,7.296365,80.736003,13.4989
2,1453659516,123,2022-10-01 03:14:03,7.296440,80.736042,13.4989
3,1453659514,123,2022-10-01 03:14:10,7.296537,80.736090,0.0000
4,1453659831,123,2022-10-01 03:15:07,7.296345,80.736147,3.7797
...,...,...,...,...,...,...
58462,1546680727,123,2022-10-31 23:09:02,7.296637,80.736007,0.0000
58463,1546680920,123,2022-10-31 23:09:26,7.296637,80.736007,0.0000
58464,1546680918,123,2022-10-31 23:09:28,7.296637,80.736007,0.0000
58465,1548465701,123,2022-10-31 23:10:26,7.296637,80.736007,0.0000


# extract trip

In [242]:

def calculate_dwell_time(row, dwell, dwell_start, dwell_time):
    if row['speed'] == 0 and not dwell:
        dwell = True
        dwell_start = row['devicetime']
    elif dwell and (row['speed'] != 0):
        dwell = False
        dwell_end = row['devicetime']
        dwell_time += (dwell_end - dwell_start).total_seconds() / 60.0
    return dwell, dwell_start, dwell_time

In [ ]:
#get the bus stop id of the current bus location
def get_bus_stop_id(point, bus_stops_gdf):
   
    for index, row in bus_stops_gdf.iterrows():
        if row['geometry'].contains(point):
            return row['stop_id']

    return None

In [ ]:
#find the arrival and departure time of each bus stop
def find_arrival_departure_time(row,time_str, start_terminal,projected_point):

    global current_trip_id,date_str, bus_stop_id,new_bus_stop, bus_stop_data
    time = datetime.strptime(time_str, "%H:%M:%S")
    speed = row['speed']


    if start_terminal == 'BT01':
        new_bus_stop_id = get_bus_stop_id(projected_point, bus_terminals_buffer[bus_terminals_buffer['direction']=='Kandy-Digana'])
    else:
        new_bus_stop_id = get_bus_stop_id(projected_point, bus_terminals_buffer[bus_terminals_buffer['direction']=='Digana-Kandy'])
    

    if (bus_stop_id != new_bus_stop_id) and new_bus_stop_id:
        bus_stop_id = new_bus_stop_id
        new_bus_stop = {
            'trip_id': current_trip_id,
            'date': date_str,
            'bus_stop_id': bus_stop_id,
            'arrival_time': time,
            'arrival_time_0': None,
            'departure_time': None,
            'dwell_time': None
        }
        if speed == 0:
            new_bus_stop['arrival_time_0'] = time

    elif (bus_stop_id == new_bus_stop_id) and new_bus_stop_id:
        if speed == 0 and (not new_bus_stop['arrival_time_0']):
            new_bus_stop['arrival_time_0'] = time
            new_bus_stop['departure_time'] = None
            
        elif speed != 0 and (not new_bus_stop['departure_time']):
            new_bus_stop['departure_time'] = time


    elif (bus_stop_id != new_bus_stop_id) and (not new_bus_stop_id):
        bus_stop_id = new_bus_stop_id
        #set departure time
        if new_bus_stop['arrival_time_0'] and (not new_bus_stop['departure_time']):
            new_bus_stop['departure_time'] = time
        elif (new_bus_stop['arrival_time']) and (not new_bus_stop['departure_time']):
            new_bus_stop['departure_time'] = new_bus_stop['arrival_time']

        #set dwell time
        if new_bus_stop['arrival_time_0']:
            new_bus_stop['dwell_time'] = new_bus_stop['departure_time'] - new_bus_stop['arrival_time_0']
            new_bus_stop['arrival_time'] = new_bus_stop['arrival_time_0']
        else:
            new_bus_stop['dwell_time'] = new_bus_stop['departure_time'] - new_bus_stop['arrival_time']
        del new_bus_stop['arrival_time_0']

        new_bus_stop['arrival_time'] = new_bus_stop['arrival_time'].time()
        new_bus_stop['departure_time'] = new_bus_stop['departure_time'].time()

        bus_stop_data = pd.concat([bus_stop_data, pd.DataFrame([new_bus_stop])])
        bus_stop_data.to_csv('bus_stop_data.csv', index=False)


In [243]:
def process_row(row, end_terminal_buffer, bus_terminals_buffer):
    
    global current_trip_id,date_str, in_trip, date, start_time, start_terminal, end_time, end_terminal, travel_time, dwell_time, inside, dwell, dwell_start,trip_data,terminal, bus_stop_id, new_bus_stop_id,arrival_time,departure_time,bus_stop_data
    row['devicetime'] = pd.to_datetime(row['devicetime'])
    date_str = row['devicetime'].strftime('%Y-%m-%d')
    time_str = row['devicetime'].strftime('%H:%M:%S')
    point_geometry = Point(row['longitude'], row['latitude'])
    projected_point = gpd.GeoSeries([point_geometry], crs='EPSG:4326').to_crs(epsg=5234)[0]

    if end_terminal_buffer.contains(projected_point).any():
        
        terminal = 'BT01' if end_terminal_buffer.iloc[0].geometry.contains(projected_point) else 'BT02'
        inside = True
        if in_trip:
            if inside:
                in_trip = False
                end_time = time_str
                end_terminal = terminal
                travel_time = (datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).total_seconds() / 60.0
                
                if (start_terminal != end_terminal) and (travel_time > 0):
                    new_trip_row = {
                        'trip_id': current_trip_id,
                        'date': date,
                        'start_time': start_time,
                        'end_time': end_time,
                        'start_terminal': start_terminal,
                        'end_terminal': end_terminal,
                        'travel_time': travel_time,
                        'dwell_time': dwell_time
                    }
                    trip_data = pd.concat([trip_data, pd.DataFrame([new_trip_row])])
                    trip_data.to_csv('trip_data.csv', index=False)
                    current_trip_id += 1

                else:
                    bus_stop_data=bus_stop_data[bus_stop_data['trip_id']!=current_trip_id]
                

                dwell_time = 0
                dwell = False

    elif not end_terminal_buffer.contains(projected_point).any() and inside:
        inside = False
        if not in_trip:
            in_trip = True
            date = date_str
            start_time = time_str
            start_terminal = terminal

    if in_trip:
        if ((start_terminal == 'BT01') and (not bus_terminals_buffer[bus_terminals_buffer['direction'] == 'Kandy-Digana'].contains(projected_point).any())) or ((start_terminal == 'BT02') and (not bus_terminals_buffer[bus_terminals_buffer['direction'] == 'Digana-Kandy'].contains(projected_point).any())):
            dwell, dwell_start, dwell_time = calculate_dwell_time(row, dwell, dwell_start, dwell_time)

        find_arrival_departure_time(row,time_str,start_terminal,projected_point)

In [244]:
trip_data = pd.read_csv('trip_data.csv')

In [245]:
# Create empty DataFrames
bus_stop_data = pd.DataFrame(columns=['trip_id','date', 'bus_stop_id','arrival_time', 'departure_time','dwell_time'])

# Initialize variables
current_trip_id = current_trip_id = trip_data.iloc[-1]['trip_id'] + 1 if len(trip_data) > 0 else 1
in_trip = False
date = None
start_time = None
start_terminal = None
end_time = None
end_terminal = None
travel_time = None
dwell_time = 0
inside = False
dwell = False
dwell_start = None
dwell_end = None
terminal = None
bus_stop_id = None
new_bus_stop_id = None
arrival_time = None
departure_time = None

# Iterate through the rows of df123_11
for index, row in df123.iterrows():
    process_row(row, end_terminal_buffer, bus_terminals_buffer)

# Print the trip_data DataFrame
print(trip_data.tail(25))


   trip_id        date start_time  end_time start_terminal end_terminal  \
0     1080  2022-10-27   07:51:41  08:41:40           BT01         BT02   
0     1081  2022-10-27   10:59:41  11:46:44           BT02         BT01   
0     1082  2022-10-27   12:31:51  13:21:02           BT01         BT02   
0     1083  2022-10-27   15:20:00  16:00:55           BT02         BT01   
0     1084  2022-10-27   16:51:56  17:36:48           BT01         BT02   
0     1085  2022-10-28   05:48:51  06:32:53           BT02         BT01   
0     1086  2022-10-28   07:30:18  08:18:19           BT01         BT02   
0     1087  2022-10-28   10:39:46  11:27:41           BT02         BT01   
0     1088  2022-10-28   12:21:58  13:13:19           BT01         BT02   
0     1089  2022-10-28   15:10:54  15:58:24           BT02         BT01   
0     1090  2022-10-28   16:52:00  17:40:36           BT01         BT02   
0     1091  2022-10-29   06:11:06  06:55:27           BT02         BT01   
0     1092  2022-10-29   

In [246]:
trip_data.to_csv('trip_data.csv', index=False)

In [247]:
# # code to map df123_11 and end_terminal_buffer using folium
# import folium
# from folium.plugins import MarkerCluster

# # Create a base map centered around the mean latitude and longitude of the data
# mean_latitude = df123_11['latitude'].mean()
# mean_longitude = df123_11['longitude'].mean()
# m = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=15)

# # Create a MarkerCluster for df123_11
# marker_cluster = MarkerCluster().add_to(m)

# # Add markers for each row in df123_11
# for index, row in df123_11.iterrows():
#     folium.Marker(
#         location=[row['latitude'], row['longitude']],
#         icon=None,
#         popup=f"Device ID: {row['deviceid']}<br>Time: {row['devicetime']}"
#     ).add_to(marker_cluster)

# # Add the end_terminal_buffer as a GeoJSON layer
# folium.GeoJson(
#     end_terminal_buffer,
#     name='Terminal Buffer',
#     style_function=lambda x: {'fillColor': 'blue', 'fillOpacity': 0.2}
# ).add_to(m)

# # Display the map
# m.save('trip_map.html')

